In [1]:
from glob import glob
from dateutil import parser
from typing import Union, Optional
import uproot as ur
import numpy as np
import json, zhh

version = 'v1'
REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/Preselection/{version}'
ILD_VERSION = 'ILD_l5_o1_v02'
PROD_NAME = '500-TDR_ws'

In [2]:
metafile = glob(f'{DATA_ROOT}/htcondor_jobs*.json')[0]
with open(metafile) as file:
    meta = json.load(file)

In [3]:
meta

{'jobs': {'1': {'job_id': 'dummy_job_id',
   'branches': [0],
   'status': 'finished',
   'code': 0,
   'error': None,
   'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_0To1.txt'}},
  '2': {'job_id': 'dummy_job_id',
   'branches': [1],
   'status': 'finished',
   'code': 0,
   'error': None,
   'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_1To2.txt'}},
  '3': {'job_id': 'dummy_job_id',
   'branches': [2],
   'status': 'finished',
   'code': 0,
   'error': None,
   'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_2To3.txt'}},
  '4': {'job_id': 'dummy_job_id',
   'branches': [3],
   'status': 'finished',
   'code': 0,
   'error': None,
   'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_3To4.txt'}},
  '5': {'job_id': 'dummy_job_id',
   'branches': [4],
   'status': 'finished',
   'code': 0,
   'error': None,
   'extra': {'log': '/nfs/dust/ilc/user/bliewert/zhh/Preselection/v1/stdall_4To5.

In [4]:
def evaluate(branch:Union[int,str]):
    branch = int(branch)
    
    with open(f'{DATA_ROOT}/{branch}_Source.txt') as file:
        src_path = file.read().strip()
        process = src_path.split(f'/{PROD_NAME}/')[1]
        process = process.split(f'/{ILD_VERSION}/')[0]
        
    with open(f'{DATA_ROOT}/stdall_{branch}To{branch+1}.txt') as file:
        signals = ['start time    :', 'end time      :', 'job exit code :']
        values = ['', '', '']
        lsig = len(signals)
        
        for line in file.readlines():
            for i in range(lsig):
                if line.startswith(signals[i]):
                    values[i] = line.split(f'{signals[i]} ')[1].strip()
                    
        for i in [0, 1]:
            if values[i] != '':
                if ' (' in values[i]:
                    values[i] = values[i].split(' (')[0]
                
                values[i] = float(parser.parse(values[i]).timestamp())
            
    return (process, src_path, values[0], values[1], values[1] - values[0], int(values[2]))

evaluate(0)

('2f_Z_bhabhaNg',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250126.P2f_z_bhabhang.eL.pL.n001.d_dstm_10602_0.slcio',
 1721334662.805879,
 1721334803.618228,
 140.81234884262085,
 0)

In [5]:
def get_results(meta)->np.ndarray:
    """_summary_

    Args:
        meta (_type_): _description_

    Returns:
        np.ndarray: _description_
    """
    
    jobs = meta['jobs']
    dtype = [
        ('process', '<U32'),
        ('src', '<U255'),
        ('tStart', 'f'),
        ('tEnd', 'f'),
        ('tDuration', 'f'),
        ('exitCode', 'i')]
    
    results = np.empty(0, dtype=dtype)

    for job_key in jobs:
        branch = jobs[job_key]['branches'][0]
        if jobs[job_key]['status'] == 'finished':
            ev = evaluate(branch)
            results = np.append(results, np.array([ev], dtype=dtype))
    
    return results

In [6]:
results = get_results(meta)

In [7]:
results[results['tDuration'] > 1800]

array([('4f_ZZ_hadronic', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/4f_ZZ_hadronic/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250002.P4f_zz_h.eL.pR.n001.d_dstm_10402_0.slcio', 1.7213347e+09, 1.7213371e+09, 2404.9563, 0),
       ('6f_ttbar', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/6f_ttbar/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I108655.Pyyveev.eL.pL.n001.d_dstm_10271_0.slcio', 1.7213347e+09, 1.7213367e+09, 1979.9971, 0),
       ('6f_ttbar', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/6f_ttbar/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I108656.Pyyveev.eL.pR.n001.d_dstm_10271_0.slcio', 1.7213347e+09, 1.7213376e+09, 2900.337 , 0),
       ('6f_ttbar', '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/6f_ttbar/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I108657.Pyyveev.eR.pL.n001.d_dstm_10271_0.slcio', 1.721334

Final State Analysis

In [8]:
branch = 0
with ur.open(f'f'{DATA_ROOT}/{branch}_FinalStates.root'') as rf:
    errors = rf['eventTree']['preselPassed'].array()